In [111]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely.geometry as geom
from fiona.crs import from_epsg
from itertools import product

In [2]:
hexagon = gpd.GeoDataFrame.from_file('/Users/nnhoucheng/Taxi_Relief_Stand_Spark/Hexagon_clipped.geojson')

In [171]:
hexagon.head()

,GRID_ID,geometry,id
0,BZ-45,"POLYGON ((1014951.499084486 205223.478088384, ...",0
1,CC-66,"POLYGON ((1018899.721130365 172550.5435180648,...",0
2,CF-32,"POLYGON ((1023153.727822219 225508.8404002782,...",0
3,BG-76,"POLYGON ((989946.0928955073 157353.829895009, ...",0
4,CP-44,"POLYGON ((1036008.683288587 206743.149475106, ...",0


In [12]:
manhattan_id = pd.read_csv('manhattan_grids.csv', index_col=0)['0'].values

In [39]:
hexagon_manhattan = hexagon[hexagon.GRID_ID.isin(manhattan_id)].reset_index(drop=True)
hexagon_manhattan['centroid'] = hexagon_manhattan.geometry.centroid

In [51]:
def distance(p):
    p1, p2 = p
    return np.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

In [28]:
p1 = hexagon_manhattan[hexagon_manhattan.GRID_ID == 'BE-42'].geometry.centroid.values[0]

In [29]:
p2 = hexagon_manhattan[hexagon_manhattan.GRID_ID == 'BE-43'].geometry.centroid.values[0]

In [78]:
distance((p1,p2))

1519.671393502882

In [79]:
def check_d(dis):
    if dis == 0:
        return 0
    elif dis <= 1521:
        return 1
    else:
        return 0

In [80]:
matrix = map(distance,product(hexagon_manhattan.centroid, hexagon_manhattan.centroid))
matrix = np.array(list(map(check_d, matrix))).reshape(432,432)

In [81]:
np.sum(matrix, axis=0)

array([3, 2, 6, 6, 0, 6, 6, 6, 3, 3, 6, 6, 4, 4, 6, 3, 2, 6, 6, 6, 6, 3, 4,
       3, 2, 6, 6, 2, 6, 2, 2, 3, 6, 6, 6, 6, 4, 5, 3, 6, 3, 0, 3, 6, 5, 1,
       2, 6, 3, 5, 6, 4, 6, 4, 5, 6, 6, 6, 1, 3, 6, 6, 6, 2, 6, 6, 6, 6, 3,
       3, 5, 5, 6, 3, 6, 3, 6, 6, 2, 5, 0, 6, 6, 6, 4, 3, 3, 6, 5, 2, 6, 6,
       6, 3, 6, 6, 2, 2, 6, 6, 6, 3, 5, 6, 6, 1, 6, 3, 5, 6, 2, 6, 2, 4, 6,
       6, 3, 6, 3, 6, 5, 1, 6, 6, 6, 5, 4, 3, 6, 3, 3, 4, 6, 3, 3, 1, 6, 3,
       2, 5, 4, 0, 6, 6, 6, 6, 5, 6, 6, 6, 6, 4, 3, 6, 2, 2, 1, 5, 4, 6, 3,
       3, 6, 6, 6, 6, 5, 3, 5, 6, 4, 6, 5, 6, 3, 2, 6, 5, 6, 5, 4, 6, 3, 6,
       1, 3, 6, 3, 1, 5, 6, 4, 6, 5, 6, 6, 5, 3, 6, 6, 5, 6, 3, 6, 6, 4, 2,
       2, 3, 5, 4, 6, 4, 2, 3, 3, 6, 4, 3, 6, 3, 2, 6, 4, 6, 6, 6, 4, 3, 6,
       6, 6, 4, 6, 6, 2, 6, 1, 6, 6, 3, 3, 6, 6, 6, 6, 6, 1, 6, 5, 6, 3, 6,
       5, 6, 6, 0, 2, 6, 3, 6, 6, 6, 5, 5, 6, 6, 6, 6, 3, 5, 0, 2, 2, 4, 6,
       3, 6, 6, 3, 6, 3, 6, 5, 6, 6, 3, 2, 6, 3, 6, 6, 6, 2, 4, 6, 6, 6, 5,
       6, 3,

In [88]:
np.where(np.sum(matrix, axis=0) == 0)[0]

array([  4,  41,  80, 141, 256, 271, 331, 405, 406])

In [89]:
hexagon_manhattan.GRID_ID[np.where(np.sum(matrix, axis=0) == 0)[0]]

4      BK-44
41     BP-36
80     BT-15
141     BS-9
256    BS-16
271    BK-46
331    BV-12
405    BG-51
406    BR-18
Name: GRID_ID, dtype: object

In [94]:
manhattan_id_final = set(manhattan_id) - set(hexagon_manhattan.GRID_ID[np.where(np.sum(matrix, axis=0) == 0)[0]])

In [97]:
pd.DataFrame(list(manhattan_id_final)).to_csv('manhattan_grids_final.csv')

# Final

In [103]:
hexagon_manhattan = hexagon[hexagon.GRID_ID.isin(manhattan_id_final)].reset_index(drop=True)
hexagon_manhattan['centroid'] = hexagon_manhattan.geometry.centroid

In [104]:
matrix = map(distance,product(hexagon_manhattan.centroid, hexagon_manhattan.centroid))
matrix = np.array(list(map(check_d, matrix))).reshape(423,423)

In [108]:
np.save('neighbour.npy', matrix)

## Request

In [115]:
request_dat = gpd.GeoDataFrame.from_file("stand_requests/stand_requests.shp")
request_dat = request_dat.set_geometry("geometry")
request_dat.crs = from_epsg(4326)
request_dat = request_dat.to_crs(epsg=2263)

In [117]:
request_dat = request_dat[request_dat.borough == 'Manhattan']

In [118]:
request_dat

,borough,cartodb_id,dot_reques,geometry,lat,location,long,request_no,side_of_st,stand_type
5,Manhattan,6,2016-12-13,POINT (992245.7025136991 224493.0037671806),40.782854,Central Park Westb/w 81st Street and 83rd Street,-73.971128,Top 10 recommendations,N or S,FHV/Taxi Relief Stand
6,Manhattan,7,2016-12-13,POINT (991808.3901657631 215697.8566142494),40.758714,Park Avenue b/w 57th Street and 51st Street,-73.972717,Top 10 recommendations,N or S,FHV/Taxi Relief Stand
7,Manhattan,8,2016-12-13,POINT (992255.2704169294 216476.5790030815),40.760851,Park Avenue b/w 57th Street and 51st Street,-73.971103,Top 10 recommendations,N or S,FHV/Taxi Relief Stand
8,Manhattan,9,2016-12-13,POINT (988560.1314406431 204798.0735094974),40.728799,1st Avenue b/w 11th Street and 9th Street,-73.984449,Top 10 recommendations,N or S,FHV/Taxi Relief Stand
9,Manhattan,10,2016-12-13,POINT (984587.6112602645 202704.6117430017),40.723054,Broadway Prince Street and Broome Street,-73.998782,Top 10 recommendations,N or S,FHV/Taxi Relief Stand
10,Manhattan,11,2016-12-13,POINT (980654.6532773225 195344.3093272524),40.702851,Broadway Beaver Street and Water Street,-74.012967,Top 10 recommendations,N or S,FHV/Taxi Relief Stand
11,Manhattan,12,2016-12-13,POINT (980557.4337903388 196014.6909668889),40.704691,Broadway Beaver Street and Water Street,-74.013318,Top 10 recommendations,N or S,FHV/Taxi Relief Stand
18,Manhattan,19,2016-12-13,POINT (1000834.28874055 226979.9286140759),40.789668,106th Streetb/w FDR Drive and 2nd Avenue,-73.940109,Additional recommendation for new relief stands,N or S,FHV/Taxi Relief Stand
19,Manhattan,20,2016-12-13,POINT (994092.1008055966 232522.8895537063),40.804892,112th Streetb/w Broadway and Amsterdam,-73.964449,Additional recommendation for new relief stands,N or S,FHV/Taxi Relief Stand
20,Manhattan,21,2016-12-13,POINT (997803.093404907 240146.5871629955),40.825812,144th Streetb/w Riverside and St. Nicholas Avenue,-73.951029,Additional recommendation for new relief stands,N or S,FHV/Taxi Relief Stand


In [151]:
def find_id(p):
    for i in hexagon_manhattan.index.values:
        if hexagon_manhattan.loc[i, 'geometry'].contains(p):
            return hexagon_manhattan.loc[i, 'GRID_ID']
    return None

In [121]:
req_id = request_dat.geometry.apply(find_id)

In [123]:
np.save('req_id.npy', np.array(req_id))

## Usage

In [125]:
usage = pd.read_csv('/Users/nnhoucheng/Taxi_Relief_Stand_Spark/usage_2015_July23.csv')

In [139]:
relief = gpd.GeoDataFrame.from_file('/Users/nnhoucheng/Taxi_Relief_Stand_Spark/relief_stands_23July.geojson')
relief = relief.set_geometry("geometry")
relief.crs = from_epsg(4326)
relief = relief.to_crs(epsg=2263)

In [129]:
usage_annual = usage.groupby('relief_stand_idx').agg({'car_usage': 'mean',
                                                      'time_usage': 'mean'})
usage_annual.reset_index(inplace=True)

In [152]:
relief_id = relief.geometry.apply(find_id)

In [158]:
usage_annual

,relief_stand_idx,time_usage,car_usage
0,0,612823.747945,76.764384
1,1,268156.501370,28.268493
2,2,5291.157303,1.202247
3,3,5912.886598,1.175258
4,4,36275.443526,7.964187
5,5,44927.545205,6.323288
6,6,14867.017986,1.852518
7,7,31610.483333,4.947222
8,8,13577.113772,3.116766
9,9,7583.954887,1.218045


In [159]:
hex_usage = {}
for i,idx in enumerate(relief_id):
    if idx:
        u = hex_usage.get(idx, (0,0))
        hex_usage[idx] = (u[0]+usage_annual.loc[i,'time_usage'], u[1]+usage_annual.loc[i,'car_usage'])

In [167]:
df_hex_usage = pd.DataFrame(hex_usage).transpose()
df_hex_usage.reset_index(inplace=True)
df_hex_usage.rename(columns={'index': 'GRID_ID',
                             0: 'time_usage',
                             1: 'car_usage'}, inplace=True)

In [169]:
df_hex_usage.to_csv('hex_usage.csv')